### Get reverse geocoding

Going through the EXIF dataframe, query the Google Maps API for geo information (country, state(s), city)

In [ ]:
# Python modules
import googlemaps
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

# Import additional functions
from flickr_functions import df_remove_dupes

# Keys needed for API access
api_key = os.getenv('maps_api_key')

# Configure maps access
gmaps = googlemaps.Client(key=api_key)

# Data directory used to store CVS files
data_dir = './data/'

In [ ]:
df_photo_exif = pd.read_csv(data_dir + 'df_photo_exif_final.csv', index_col=[0])
i = 0
for index, row in df_photo_exif.iterrows():

    photo_id = int(row.get('id'))
    lat = row.get('lat')
    lon = row.get('lon')
    country, admin_lvl1, admin_lvl2, city = '', '', '', ''
    if row.get('lat') != 'na' and row.get('country') == 'tbd':
        i += 1
print(i)
# 39217
# 39231
# 39078
# 22197
# 9893

In [ ]:
# Load EXIF dataframe from CVS file
df_photo_exif = pd.read_csv(data_dir + 'df_photo_exif_final.csv', index_col=[0])

# Create columns if they don't exist
if 'country' not in df_photo_exif.columns:
    df_photo_exif['country'] = 'tbd'
if 'admin_lvl1' not in df_photo_exif.columns:
    df_photo_exif['admin_lvl1'] = 'tbd'
if 'admin_lvl2' not in df_photo_exif.columns:
    df_photo_exif['admin_lvl2'] = 'tbd'
if 'city' not in df_photo_exif.columns:
    df_photo_exif['city'] = 'tbd'

df_tmp = pd.DataFrame(columns = ['id', 'country', 'admin_lvl1', 'admin_lvl2', 'city'])

counter = 0

# Iterate through dataframe and see if there's a value for lat and lon
for index, row in df_photo_exif.iterrows():

    photo_id = int(row.get('id'))
    lat = row.get('lat')
    lon = row.get('lon')
    country, admin_lvl1, admin_lvl2, city = '', '', '', ''
    
    # Check if there is latitude and if cell has not been filled yet
    if lat != 'na' and row.get('country') == 'tbd':

        print(f'Looking up photo with lat/lon: {lat}/{lon}')
    
        # Query the API with the location data
        geo = gmaps.reverse_geocode((lat, lon))
    
        # Get the data from the response
        for comp in geo[0].get('address_components'):
            if 'country' in comp.get('types'):
                country = comp.get('long_name')
            if 'administrative_area_level_1' in comp.get('types'):
                admin_lvl1 = comp.get('long_name')
            if 'administrative_area_level_2' in comp.get('types'):
                admin_lvl2 = comp.get('long_name')
            if 'locality' in comp.get('types'):
                city = comp.get('long_name')
            elif 'postal_town' in comp.get('types'):
                city = comp.get('long_name')

        print(counter, country, admin_lvl1, admin_lvl2, city)
    
        # Add the data to the dataframe
        df_photo_exif.loc[index, 'country'] = country
        df_photo_exif.loc[index, 'admin_lvl1'] = admin_lvl1
        df_photo_exif.loc[index, 'admin_lvl2'] = admin_lvl2
        df_photo_exif.loc[index, 'city'] = city

        counter += 1

In [ ]:
df_photo_exif.to_csv(data_dir + 'df_photo_exif_final.csv')

In [ ]:
df_photo_exif.head(20)